In [5]:
# for the future, create EBITDA metric for all stocks, show highest and lowest.

import yfinance as yf

company = input("Enter stock ticker (e.g. AAPL): ").upper()
ticker = yf.Ticker(company)

try:
    price = ticker.info['currentPrice']
    print(f"Live price of {company}: ${price}")
except Exception as e:
    print("Something went wrong:", e)

Enter stock ticker (e.g. AAPL):  AAPL


Live price of AAPL: $190.0519


In [3]:
#Finally working on EBITDA marker

import yfinance as yf
import pandas as pd

# Get S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(url)[0]
tickers = sp500['Symbol'].tolist()

# Fix tickers with "." (BRK.B -> BRK-B for yfinance)
tickers = [ticker.replace('.', '-') for ticker in tickers]

results = []

for ticker in tickers:
    try:
        print(f"Scanning {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        ebitda = info.get('ebitda')
        if ebitda:
            results.append({'Ticker': ticker, 'EBITDA': ebitda})
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")


# Convert and sort
df = pd.DataFrame(results)
df = df.sort_values(by='EBITDA', ascending=False).reset_index(drop=True)

# print("\nTop 10 companies by EBITDA:")
# print(df.head(10))

# print("\nBottom 10 companies by EBITDA:")
# print(df.tail(10))

# Save to Excel
df.to_excel("ebitda_sp500.xlsx", index=False)

Scanning MMM...
Scanning AOS...
Scanning ABT...
Scanning ABBV...
Scanning ACN...
Scanning ADBE...
Scanning AMD...
Scanning AES...
Scanning AFL...
Scanning A...
Scanning APD...
Scanning ABNB...
Scanning AKAM...
Scanning ALB...
Scanning ARE...
Scanning ALGN...
Scanning ALLE...
Scanning LNT...
Scanning ALL...
Scanning GOOGL...
Scanning GOOG...
Scanning MO...
Scanning AMZN...
Scanning AMCR...
Scanning AEE...
Scanning AEP...
Scanning AXP...
Scanning AIG...
Scanning AMT...
Scanning AWK...
Scanning AMP...
Scanning AME...
Scanning AMGN...
Scanning APH...
Scanning ADI...
Scanning ANSS...
Scanning AON...
Scanning APA...
Scanning APO...
Scanning AAPL...
Scanning AMAT...
Scanning APTV...
Scanning ACGL...
Scanning ADM...
Scanning ANET...
Scanning AJG...
Scanning AIZ...
Scanning T...
Scanning ATO...
Scanning ADSK...
Scanning ADP...
Scanning AZO...
Scanning AVB...
Scanning AVY...
Scanning AXON...
Scanning BKR...
Scanning BALL...
Scanning BAC...
Scanning BAX...
Scanning BDX...
Scanning BRK-B...
Scanni

In [8]:
#My metrics based on EBITDA: A strong EBITDA to Interest Expense Ratio and a reasonable stock price relative to their market cap, would score the highest.

import yfinance as yf
import pandas as pd

# Get S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(url)[0]
tickers = sp500['Symbol'].tolist()

# Fix tickers with "." (BRK.B -> BRK-B for yfinance)
tickers = [ticker.replace('.', '-') for ticker in tickers]
metrics = []

for ticker in tickers:
    try:
        print(f"Scanning {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Get required metrics (with default values if not present)
        ebitda = info.get('ebitda', None)
        interest_expense = info.get('interestExpense', None)
        market_cap = info.get('marketCap', None)
        
# Check if we have all necessary data. Otherwise we have huge problems.
        if ebitda is not None and interest_expense is not None and market_cap is not None:
            ebitda_to_interest_expense = ebitda / interest_expense
        else:
# Default behavior for missing values
            ebitda_to_interest_expense = None 
            
        metrics.append({
            'Ticker': ticker,
            'EBITDA': ebitda,
            'Interest Expense': interest_expense,
            'Market Cap': market_cap,
            'EBITDA to Interest Expense Ratio': ebitda_to_interest_expense
        })
        
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

# Check if metrics have been collected
print(f"Number of valid companies: {len(metrics)}")

# Convert results to DataFrame if we have valid data
if len(metrics) > 0:
    df = pd.DataFrame(metrics)

    print("Columns in DataFrame:")
    print(df.columns)

    # Rank companies based on EBITDA to Interest Expense Ratio and Market Cap
    df['Rank'] = df['EBITDA to Interest Expense Ratio'] * 1000 / df['Market Cap']

    # Sort the companies by rank
    df = df.sort_values(by='Rank', ascending=False).reset_index(drop=True)

    # Show the top-ranked companies
    print(df.head(10))

    # Save to Excel
    df.to_excel("ebitda_analysis_sp500.xlsx", index=False)
else:
    print("No valid data to process.")

Scanning MMM...
Scanning AOS...
Scanning ABT...
Scanning ABBV...
Scanning ACN...
Scanning ADBE...
Scanning AMD...
Scanning AES...
Scanning AFL...
Scanning A...
Scanning APD...
Scanning ABNB...
Scanning AKAM...
Scanning ALB...
Scanning ARE...
Scanning ALGN...
Scanning ALLE...
Scanning LNT...
Scanning ALL...
Scanning GOOGL...
Scanning GOOG...
Scanning MO...
Scanning AMZN...
Scanning AMCR...
Scanning AEE...
Scanning AEP...
Scanning AXP...
Scanning AIG...
Scanning AMT...
Scanning AWK...
Scanning AMP...
Scanning AME...
Scanning AMGN...
Scanning APH...
Scanning ADI...
Scanning ANSS...
Scanning AON...
Scanning APA...
Scanning APO...
Scanning AAPL...
Scanning AMAT...
Scanning APTV...
Scanning ACGL...
Scanning ADM...
Scanning ANET...
Scanning AJG...
Scanning AIZ...
Scanning T...
Scanning ATO...
Scanning ADSK...
Scanning ADP...
Scanning AZO...
Scanning AVB...
Scanning AVY...
Scanning AXON...
Scanning BKR...
Scanning BALL...
Scanning BAC...
Scanning BAX...
Scanning BDX...
Scanning BRK-B...
Scanni

In [11]:
# Ok, how can we do it? 
import yfinance as yf

# Test ticker (replace with any ticker you want to check)
company = 'AAPL'
ticker = yf.Ticker(company)

# Fetch the stock info
info = ticker.info

# Print the whole info dictionary to inspect available fields
print(info)

# Check if 'interestExpense' exists or find any alternative data
interest_expense = info.get('interestExpense', None)
print(f"Interest Expense for {company}: {interest_expense}")

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as w

In [12]:
#Ok, based on this info lets do weighted metrics. There is a score for each stock and then stocks are ranked based on this score. Stocks with the highest scores will be ranked as the best in terms of undervaluation.

import yfinance as yf
import pandas as pd

# Get S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(url)[0]
tickers = sp500['Symbol'].tolist()

# Fix tickers with "." (BRK.B -> BRK-B for yfinance)
tickers = [ticker.replace('.', '-') for ticker in tickers]
metrics = []

for ticker in tickers:
    try:
        print(f"Scanning {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Get required metrics (with default values if not present)
        ebitda = info.get('ebitda', None)
        interest_expense = info.get('interestExpense', None)
        market_cap = info.get('marketCap', None)
        trailing_pe = info.get('trailingPE', None)
        price_to_book = info.get('priceToBook', None)
        price_to_sales = info.get('priceToSalesTrailing12Months', None)
        ev_ebitda = info.get('enterpriseToEbitda', None)
        earnings_growth = info.get('earningsGrowth', None)

# Check if we have all necessary data. Otherwise, assign None
        if ebitda is not None and interest_expense is not None and market_cap is not None:
            ebitda_to_interest_expense = ebitda / interest_expense
        else:
            ebitda_to_interest_expense = None

# Use the metrics to calculate a combined score
# Set default weights (these can be adjusted)
        weights = {
            'EBITDA to Interest Expense Ratio': 0.25,  # 25% weight
            'Trailing PE': 0.2,                       # 20% weight
            'Price to Book': 0.2,                     # 20% weight
            'Price to Sales': 0.15,                   # 15% weight
            'EV/EBITDA': 0.1,                        # 10% weight
            'Earnings Growth': 0.1                    # 10% weight
        }

        # Calculate individual scores (normalized)
        score = 0
        if ebitda_to_interest_expense is not None:
            score += ebitda_to_interest_expense * weights['EBITDA to Interest Expense Ratio']
        if trailing_pe is not None:
            score += (1 / trailing_pe) * weights['Trailing PE']  # Lower P/E is better
        if price_to_book is not None:
            score += (1 / price_to_book) * weights['Price to Book']  # Lower P/B is better
        if price_to_sales is not None:
            score += (1 / price_to_sales) * weights['Price to Sales']  # Lower P/S is better
        if ev_ebitda is not None:
            score += (1 / ev_ebitda) * weights['EV/EBITDA']  # Lower EV/EBITDA is better
        if earnings_growth is not None:
            score += earnings_growth * weights['Earnings Growth']  # Higher growth is better

        # Append the company's metrics and calculated score
        metrics.append({
            'Ticker': ticker,
            'EBITDA': ebitda,
            'Interest Expense': interest_expense,
            'Market Cap': market_cap,
            'EBITDA to Interest Expense Ratio': ebitda_to_interest_expense,
            'Trailing P/E': trailing_pe,
            'Price to Book': price_to_book,
            'Price to Sales': price_to_sales,
            'EV/EBITDA': ev_ebitda,
            'Earnings Growth': earnings_growth,
            'Score': score
        })
        
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

# Check if metrics have been collected
print(f"Number of valid companies: {len(metrics)}")

# Convert results to DataFrame if we have valid data
if len(metrics) > 0:
    df = pd.DataFrame(metrics)

    print("Columns in DataFrame:")
    print(df.columns)

    # Rank companies based on the combined score
    df['Rank'] = df['Score']

    # Sort the companies by rank (higher score = better)
    df = df.sort_values(by='Rank', ascending=False).reset_index(drop=True)

    # Show the top-ranked companies
    print(df.head(10))

    # Save to Excel
    df.to_excel("combined_analysis_sp500.xlsx", index=False)
else:
    print("No valid data to process.")

In [13]:
#We also need to change how EBITDA to expense ratio is calculated based on what we have

import yfinance as yf
import pandas as pd

# Get S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(url)[0]
tickers = sp500['Symbol'].tolist()

# Fix tickers with "." (BRK.B -> BRK-B for yfinance)
tickers = [ticker.replace('.', '-') for ticker in tickers]
metrics = []

# Assumed average interest rate for debt (this can be adjusted based on sector averages)
average_interest_rate = 0.05  # 5%

# Set default weights (these can be adjusted)
weights = {
    'EBITDA to Interest Expense Ratio': 0.25,  # 25% weight
    'Trailing PE': 0.2,                       # 20% weight
    'Price to Book': 0.2,                     # 20% weight
    'Price to Sales': 0.15,                   # 15% weight
    'EV/EBITDA': 0.1,                        # 10% weight
    'Earnings Growth': 0.1                    # 10% weight
}

for ticker in tickers:
    try:
        print(f"Scanning {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Get required metrics (with default values if not present)
        ebitda = info.get('ebitda', None)
        total_debt = info.get('totalDebt', None)
        market_cap = info.get('marketCap', None)
        trailing_pe = info.get('trailingPE', None)
        price_to_book = info.get('priceToBook', None)
        price_to_sales = info.get('priceToSalesTrailing12Months', None)
        ev_ebitda = info.get('enterpriseToEbitda', None)
        earnings_growth = info.get('earningsGrowth', None)
        
        # Estimate Interest Expense using Total Debt and average interest rate
        if total_debt is not None:
            interest_expense = total_debt * average_interest_rate
        else:
            interest_expense = None
        
        # Calculate EBITDA to Interest Expense ratio if we have both values
        if ebitda is not None and interest_expense is not None and market_cap is not None:
            ebitda_to_interest_expense = ebitda / interest_expense
        else:
            ebitda_to_interest_expense = None

        # Calculate individual scores (normalized)
        score = 0
        if ebitda_to_interest_expense is not None:
            score += ebitda_to_interest_expense * weights['EBITDA to Interest Expense Ratio']
        if trailing_pe is not None:
            score += (1 / trailing_pe) * weights['Trailing PE']  # Lower P/E is better
        if price_to_book is not None:
            score += (1 / price_to_book) * weights['Price to Book']  # Lower P/B is better
        if price_to_sales is not None:
            score += (1 / price_to_sales) * weights['Price to Sales']  # Lower P/S is better
        if ev_ebitda is not None:
            score += (1 / ev_ebitda) * weights['EV/EBITDA']  # Lower EV/EBITDA is better
        if earnings_growth is not None:
            score += earnings_growth * weights['Earnings Growth']  # Higher growth is better

        # Append the company's metrics and calculated score
        metrics.append({
            'Ticker': ticker,
            'EBITDA': ebitda,
            'Interest Expense': interest_expense,
            'Market Cap': market_cap,
            'EBITDA to Interest Expense Ratio': ebitda_to_interest_expense,
            'Trailing P/E': trailing_pe,
            'Price to Book': price_to_book,
            'Price to Sales': price_to_sales,
            'EV/EBITDA': ev_ebitda,
            'Earnings Growth': earnings_growth,
            'Score': score
        })
        
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

# Check if metrics have been collected
print(f"Number of valid companies: {len(metrics)}")

# Convert results to DataFrame if we have valid data
if len(metrics) > 0:
    df = pd.DataFrame(metrics)

    print("Columns in DataFrame:")
    print(df.columns)

    # Rank companies based on the combined score
    df['Rank'] = df['Score']

    # Sort the companies by rank (higher score = better)
    df = df.sort_values(by='Rank', ascending=False).reset_index(drop=True)

    # Show the top-ranked companies
    print(df.head(10))

    # Save to Excel
    df.to_excel("combined_analysis_sp500_with_estimated_interest_expense.xlsx", index=False)
else:
    print("No valid data to process.")

Scanning MMM...
Scanning AOS...
Scanning ABT...
Scanning ABBV...
Scanning ACN...
Scanning ADBE...
Scanning AMD...
Scanning AES...
Scanning AFL...
Scanning A...
Scanning APD...
Scanning ABNB...
Scanning AKAM...
Scanning ALB...
Scanning ARE...
Scanning ALGN...
Scanning ALLE...
Scanning LNT...
Scanning ALL...
Scanning GOOGL...
Scanning GOOG...
Scanning MO...
Scanning AMZN...
Scanning AMCR...
Scanning AEE...
Scanning AEP...
Scanning AXP...
Scanning AIG...
Scanning AMT...
Scanning AWK...
Scanning AMP...
Scanning AME...
Scanning AMGN...
Scanning APH...
Scanning ADI...
Scanning ANSS...
Scanning AON...
Scanning APA...
Scanning APO...
Scanning AAPL...
Scanning AMAT...
Scanning APTV...
Scanning ACGL...
Scanning ADM...
Scanning ANET...
Scanning AJG...
Scanning AIZ...
Scanning T...
Scanning ATO...
Scanning ADSK...
Scanning ADP...
Scanning AZO...
Scanning AVB...
Scanning AVY...
Scanning AXON...
Scanning BKR...
Scanning BALL...
Scanning BAC...
Scanning BAX...
Scanning BDX...
Scanning BRK-B...
Scanni

In [18]:
# Ok, now we also need to include income ratio, earnings growth rate, P/E, P/B, and P/S ratios checking whether the stock is overvalued.

import yfinance as yf
import pandas as pd

# Get S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(url)[0]
tickers = sp500['Symbol'].tolist()

# Fix tickers with "." (BRK.B -> BRK-B for yfinance)
tickers = [ticker.replace('.', '-') for ticker in tickers]
metrics = []

# Assumed average interest rate for debt (this can be adjusted based on sector averages)
average_interest_rate = 0.05  # 5%

# Set default weights (these can be adjusted)
weights = {
    'EBITDA to Interest Expense Ratio': 0.15,   # 15% weight
    'Trailing PE': 0.10,                        # 10% weight
    'Price to Book': 0.10,                      # 10% weight
    'Price to Sales': 0.10,                     # 10% weight
    'EV/EBITDA': 0.15,                         # 15% weight
    'Earnings Growth': 0.15,                    # 15% weight
    'Debt to Equity': 0.10,                     # 10% weight
    'Quick Ratio': 0.05,                        # 5% weight
    'Operating Margin': 0.10,                   # 10% weight
    'Net Profit Margin': 0.10,                  # 10% weight
    'Return on Assets': 0.05,                   # 5% weight
    'Return on Equity': 0.05                    # 5% weight
}

for ticker in tickers:
    try:
        print(f"Scanning {ticker}...")
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Get required metrics (with default values if not present)
        ebitda = info.get('ebitda', None)
        total_debt = info.get('totalDebt', None)
        market_cap = info.get('marketCap', None)
        trailing_pe = info.get('trailingPE', None)
        price_to_book = info.get('priceToBook', None)
        price_to_sales = info.get('priceToSalesTrailing12Months', None)
        ev_ebitda = info.get('enterpriseToEbitda', None)
        
        # Estimate Interest Expense using Total Debt and average interest rate
        if total_debt is not None and total_debt > 0:
            interest_expense = total_debt * average_interest_rate
        else:
            interest_expense = 0  # Default value if no debt data or zero debt
        
        # Handle missing earningsGrowth
        earnings_growth = info.get('earningsGrowth', None)
        if earnings_growth is None:
            net_income = info.get('netIncomeToCommon', None)
            previous_net_income = info.get('netIncome', None)
            if net_income is not None and previous_net_income is not None:
                earnings_growth = (net_income - previous_net_income) / previous_net_income
            else:
                earnings_growth = 0  # Default value
        
        # Handle missing Dividend Yield or Payout Ratio
        dividend_yield = info.get('dividendYield', None)
        payout_ratio = info.get('payoutRatio', None)
        
        # Calculate EBITDA to Interest Expense ratio if we have both values and avoid division by zero
        if ebitda is not None and interest_expense is not None and interest_expense > 0:
            ebitda_to_interest_expense = ebitda / interest_expense
        else:
            ebitda_to_interest_expense = None  # Set to None if not calculable
        
        # Calculate the score based on the available data
        score = 0
        if ebitda_to_interest_expense is not None:
            score += ebitda_to_interest_expense * weights['EBITDA to Interest Expense Ratio']
        if trailing_pe is not None and trailing_pe > 0:
            score += (1 / trailing_pe) * weights['Trailing PE']
        if price_to_book is not None and price_to_book > 0:
            score += (1 / price_to_book) * weights['Price to Book']
        if price_to_sales is not None and price_to_sales > 0:
            score += (1 / price_to_sales) * weights['Price to Sales']
        if ev_ebitda is not None and ev_ebitda > 0:
            score += (1 / ev_ebitda) * weights['EV/EBITDA']
        if earnings_growth is not None:
            score += earnings_growth * weights['Earnings Growth']
        
        # Add metrics to list
        metrics.append({
            'Ticker': ticker,
            'EBITDA': ebitda,
            'Interest Expense': interest_expense,
            'Market Cap': market_cap,
            'EBITDA to Interest Expense Ratio': ebitda_to_interest_expense,
            'Trailing P/E': trailing_pe,
            'Price to Book': price_to_book,
            'Price to Sales': price_to_sales,
            'EV/EBITDA': ev_ebitda,
            'Earnings Growth': earnings_growth,
            'Score': score,
            'Dividend Yield': dividend_yield,
            'Payout Ratio': payout_ratio
        })
        
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        # Add a placeholder for missing values
        metrics.append({
            'Ticker': ticker,
            'EBITDA': None,
            'Interest Expense': None,
            'Market Cap': None,
            'EBITDA to Interest Expense Ratio': None,
            'Trailing P/E': None,
            'Price to Book': None,
            'Price to Sales': None,
            'EV/EBITDA': None,
            'Earnings Growth': None,
            'Score': None,
            'Dividend Yield': None,
            'Payout Ratio': None
        })

# Check if metrics have been collected
if len(metrics) > 0:
    df = pd.DataFrame(metrics)
    df['Rank'] = df['Score']
    df = df.sort_values(by='Rank', ascending=False).reset_index(drop=True)
    print(df.head(10))
    df.to_excel("combined_analysis_sp500_with_estimated_interest_expense.xlsx", index=False)
else:
    print("No valid data to process.")

Scanning MMM...
Scanning AOS...
Scanning ABT...
Scanning ABBV...
Scanning ACN...
Scanning ADBE...
Scanning AMD...
Scanning AES...
Scanning AFL...
Scanning A...
Scanning APD...
Scanning ABNB...
Scanning AKAM...
Scanning ALB...
Scanning ARE...
Scanning ALGN...
Scanning ALLE...
Scanning LNT...
Scanning ALL...
Scanning GOOGL...
Scanning GOOG...
Scanning MO...
Scanning AMZN...
Scanning AMCR...
Scanning AEE...
Scanning AEP...
Scanning AXP...
Scanning AIG...
Scanning AMT...
Scanning AWK...
Scanning AMP...
Scanning AME...
Scanning AMGN...
Scanning APH...
Scanning ADI...
Scanning ANSS...
Scanning AON...
Scanning APA...
Scanning APO...
Scanning AAPL...
Scanning AMAT...
Scanning APTV...
Scanning ACGL...
Scanning ADM...
Scanning ANET...
Scanning AJG...
Scanning AIZ...
Scanning T...
Scanning ATO...
Scanning ADSK...
Scanning ADP...
Scanning AZO...
Scanning AVB...
Scanning AVY...
Scanning AXON...
Scanning BKR...
Scanning BALL...
Scanning BAC...
Scanning BAX...
Scanning BDX...
Scanning BRK-B...
Scanni